Note: Be sure to use your API token and your account name.

I did two versions
- first one is only with quiskit (It is taking long time) since I'm familiar with it
- in second one I used Quantum rings 

## Quiskit version (Expirimental, just for test)

In [ ]:
import math
import random
from fractions import Fraction
import numpy as np

from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import QFT

# ---- Helper functions for the classical post–processing ----

def gcd(a, b):
    """Compute the greatest common divisor using Euclid's algorithm."""
    while b:
        a, b = b, a % b
    return a

def continued_fraction_expansion(x, Q):
    """Return the best rational approximation of x/Q via continued fraction."""
    frac = Fraction(x, Q).limit_denominator()
    return frac.numerator, frac.denominator

def modexp(a, power, N):
    """Return (a^power) mod N."""
    return pow(a, power, N)

# ---- Building the order finding circuit ----

def build_order_finding_circuit(a, N, n_count):
    """
    Construct the quantum circuit for order finding.
    a: chosen integer coprime with N.
    N: number to factor.
    n_count: number of qubits in the counting register.
    
    The circuit has two registers:
      - Counting register: n_count qubits
      - Work register: enough qubits to represent N (we use ceil(log2(N)) qubits)
    """
    # Determine number of qubits for the work register:
    n_work = math.ceil(math.log(N, 2))
    
    # Total circuit registers:
    qc = QuantumCircuit(n_count + n_work, n_count)
    
    # Step 1: Initialize counting register in uniform superposition
    for q in range(n_count):
        qc.h(q)
        
    # Step 2: Initialize work register to |1>
    qc.x(n_count)  # Assuming the first qubit of work register represents |1>
    # Alternatively, if using more qubits in work register, initialize |0...01> appropriately.
    
    # Step 3: Apply controlled-U operations.
    # U is the modular multiplication by a mod N operator.
    # For each counting qubit, apply controlled U^(2^j)
    for q in range(n_count):
        # Exponent for the unitary:
        exponent = 2**q
        # We assume a modular multiplication operator U_a^(2^q) is implemented as a gate.
        U_gate = modular_exponentiation_gate(a, exponent, N, n_work)
        qc.append(U_gate.control(), [q] + list(range(n_count, n_count+n_work)))
    
    # Step 4: Apply the inverse Quantum Fourier Transform on the counting register
    qc.append(QFT(n_count, inverse=True, do_swaps=True), list(range(n_count)))
    
    # Step 5: Measure the counting register
    qc.measure(range(n_count), range(n_count))
    
    return qc

def modular_exponentiation_gate(a, exponent, N, n_work):
    """
    This is a placeholder for a gate that implements the modular exponentiation operation:
      |y> --> |a^(exponent)*y mod N>
    For simulation purposes, we use a custom gate.
    
    Here we use a QuantumCircuit and add a barrier to indicate where the actual implementation would be.
    """
    qc = QuantumCircuit(n_work, name=f"ModExp(a={a}, exp={exponent})")
    qc.barrier()
    return qc.to_gate()

# ---- Order finding subroutine ----

def run_order_finding(a, N, n_count=8):
    """
    Runs the order finding circuit for a given a and N.
    n_count: number of qubits in the counting register (precision of the phase estimation).
    Returns the measured value (as integer) and the total Q value (2^n_count).
    """
    qc = build_order_finding_circuit(a, N, n_count)
    
    # Use the Quantum Rings Simulator via qBraid.
    # For this example we use Qiskit's Aer simulator.
    simulator = Aer.get_backend('qasm_simulator')
    shots = 1024
    job = execute(qc, simulator, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # Pick the most frequently measured value.
    measured_str = max(counts, key=counts.get)
    measured_int = int(measured_str, 2)
    Q = 2**n_count
    return measured_int, Q

# ---- The main Shor algorithm ----

def shor_factor(N, max_attempts=5, n_count=8):
    """
    Attempt to factor the integer N using Shor's algorithm.
    Returns a factor pair (p, q) if successful; otherwise, returns None.
    """
    # First, check if N is even or a perfect power.
    if N % 2 == 0:
        return 2, N//2

    # Try up to max_attempts different random bases.
    for attempt in range(max_attempts):
        # Choose a random integer a in [2, N-1]
        a = random.randint(2, N - 1)
        # Check that a and N are coprime.
        if gcd(a, N) != 1:
            continue  # a is a non-trivial factor

        print(f"Attempt {attempt+1}: Testing a = {a}")

        # Run the order-finding subroutine
        measured, Q = run_order_finding(a, N, n_count)
        print(f"Measured value: {measured} (over Q = {Q})")
        
        # Use continued fraction to approximate the phase
        _, r = continued_fraction_expansion(measured, Q)
        print(f"Estimated order r = {r}")

        # Check if r is even and a^(r/2) is not -1 mod N
        if r % 2 != 0:
            print("r is odd, retrying...\n")
            continue

        # Compute potential factors
        exponent = r // 2
        factor_candidate = pow(a, exponent, N)
        if factor_candidate == N - 1 or factor_candidate == 1:
            print("Trivial factor encountered, retrying...\n")
            continue

        # Compute the gcd of (factor_candidate ± 1) and N to extract factors
        factor1 = gcd(factor_candidate + 1, N)
        factor2 = gcd(factor_candidate - 1, N)

        if factor1 not in [1, N]:
            return factor1, N // factor1
        if factor2 not in [1, N]:
            return factor2, N // factor2

        print("Failed to obtain non-trivial factors, retrying...\n")
    return None

# ---- Apply Shor's Algorithm on the list of semiprime numbers ----

import semiprimes

results = {}

print("Starting Shor's algorithm on the list of semiprimes...\n")
for bit_size, N in semiprimes.items():
    print(f"---\nFactoring {N} (bit-size {bit_size})")
    factors = shor_factor(N, max_attempts=10, n_count=bit_size//2 + 1)
    if factors:
        p, q = factors
        print(f"Success: {N} = {p} * {q}\n")
        results[N] = (p, q)
    else:
        print(f"Failed to factor {N} after several attempts.\n")
        
print("Final results:")
for N, (p, q) in results.items():
    print(f"{N} = {p} * {q}")


## Qunatum Rings version (submission version)

Note: I am submitting this without cheking the final result, I'm pushing this during execution, it is taking so much time..

In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor, JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math
import random
from fractions import Fraction

provider = QuantumRingsProvider(
    token=<your-token>,
    name=<your-email>
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

In [ ]:
def iqft_cct(qc, b, n):
    """
    The inverse QFT circuit.

    Args:
        qc (QuantumCircuit): The quantum circuit.
        b (QuantumRegister): The target register.
        n (int): The number of qubits in the register to use.

    Returns:
        None.
    """
    for i in range(n):
        for j in range(1, i+1):
            # For inverse transform, we have to use negative angles.
            qc.cu1(-math.pi / 2**(i - j + 1), b[j - 1], b[i])
        # Apply Hadamard after the rotations.
        qc.h(b[i])
    qc.barrier()
    return

def plot_histogram(counts, title=""):
    """
    Plots the histogram of the counts.

    Args:
        counts (dict): The dictionary containing the counts of states.
        title (str): A title for the graph.

    Returns:
        None.
    """
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    
    # Expand counts into a list of state outcomes.
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    
    plt.bar(unique, bin_counts)
    
    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    plt.title(title)
    plt.show()
    return

# ----------------- Helper Functions -----------------

def gcd(a, b):
    """Compute the greatest common divisor using Euclid's algorithm."""
    while b:
        a, b = b, a % b
    return a

def continued_fraction_expansion(x, Q):
    """
    Use a continued fraction expansion to find the best rational approximation
    for x/Q. Returns the numerator and denominator.
    """
    frac = Fraction(x, Q).limit_denominator()
    return frac.numerator, frac.denominator

def modexp(a, power, N):
    """Return (a^power) mod N."""
    return pow(a, power, N)

def modular_exponentiation_gate(a, exponent, N, n_work):
    """
    Create a placeholder gate that implements modular exponentiation:
      |y⟩ --> |a^(exponent) * y mod N⟩.
    
    In a real quantum solution, this should be constructed from reversible arithmetic gates.
    Here we create a dummy gate using QuantumRingsLib's API.
    """
    gate_circuit = QuantumCircuit(n_work, name=f"ModExp(a={a}, exp={exponent})")
    gate_circuit.barrier()
    # The actual gate decomposition would be implemented here.
    return gate_circuit.to_gate()

# ----------------- Building the Order Finding Circuit -----------------

def build_order_finding_circuit(a, N, n_count):
    """
    Constructs the order-finding circuit for a given base a and semiprime N.
    
    Registers:
      - Counting register: n_count qubits (for phase estimation)
      - Work register: n_work qubits (enough to represent N)
    
    Returns:
      A QuantumCircuit object configured for phase estimation.
    """
    # Determine the number of qubits needed for the work register.
    n_work = math.ceil(math.log(N, 2))
    
    # Create quantum registers.
    counting_reg = QuantumRegister(n_count, name="count")
    work_reg = QuantumRegister(n_work, name="work")
    classical_reg = ClassicalRegister(n_count, name="c")
    
    # Build the circuit with (n_count + n_work) qubits and n_count classical bits.
    qc = QuantumCircuit(counting_reg, work_reg, classical_reg)
    
    # Step 1: Apply Hadamard gates to the counting register.
    for qubit in counting_reg:
        qc.h(qubit)
        
    # Step 2: Initializing the work register to |1⟩.
    # Here we set the first qubit of the work register to |1⟩.
    qc.x(work_reg[0])
    
    # Step 3: Applying controlled modular exponentiation
    # For each qubit in the counting register, apply a controlled-U^(2^j) gate.
    for idx, qubit in enumerate(counting_reg):
        exponent = 2 ** idx
        U_gate = modular_exponentiation_gate(a, exponent, N, n_work)
        # Append the controlled version of the gate.
        qc.append(U_gate.control(), [qubit] + work_reg[:])
    
    # Step 4: Applying the inverse QFT on the counting registe
    iqft_cct(qc, counting_reg, n_count)
    
    # Step 5: Measure the counting register.
    qc.measure(counting_reg, classical_reg)
    
    qc.barrier()
    return qc

# ----------------- Order Finding Subroutine -----------------

def run_order_finding(a, N, n_count):
    """
    Runs the quantum order-finding circuit for base a and semiprime N.
    
    Parameters:
      - a: The base used in the modular exponentiation.
      - N: The semiprime number to factor.
      - n_count: Number of qubits in the counting register.
      
    Returns:
      A tuple (measured_value, Q) where:
        measured_value: The integer value obtained from measurement.
        Q: The total number of states in the counting register (2^n_count).
    """
    qc = build_order_finding_circuit(a, N, n_count)
    
    # Submit the job using the backend provided by QuantumRingsLib.
    job = provider.run(qc, backend=backend, shots=shots)
    job_monitor(job)
    
    # Once complete, get the results.
    result = job.result()
    counts = result.get_counts()
    
    # Plot histogram for visualization.
    plot_histogram(counts, title=f"Histogram for a={a}, N={N}")
    
    # Choose the outcome with the highest count.
    measured_str = max(counts, key=counts.get)
    measured_int = int(measured_str, 2)
    Q = 2 ** n_count
    return measured_int, Q

# ----------------- Shor's Algorithm -----------------

def shor_factor(N, max_attempts=5, n_count=8):
    """
    Attempt to factor the integer N using Shor's algorithm.
    
    Parameters:
      - N (int): The semiprime number to factor.
      - max_attempts (int): Maximum number of random bases (a) to try.
                            Shor's algorithm is probabilistic, so multiple attempts
                            may be necessary.
      - n_count (int): Number of qubits in the counting register (controls the precision
                       of phase estimation). Typically chosen based on the bit-size of N.
    
    Returns:
      A tuple (p, q) such that N = p * q if successful;
      Otherwise, returns None if no valid factorization is found after max_attempts.
    """
    # Quick check: if N is even, return 2 immediately.
    if N % 2 == 0:
        return 2, N // 2
    
    for attempt in range(max_attempts):
        # Pick a random integer a between 2 and N-1.
        a = random.randint(2, N - 1)
        
        # If a and N are not coprime, then gcd(a, N) is a non-trivial factor.
        if gcd(a, N) != 1:
            return gcd(a, N), N // gcd(a, N)
        
        print(f"Attempt {attempt + 1}: Testing a = {a}")
        
        # Run the quantum order-finding subroutine.
        measured, Q = run_order_finding(a, N, n_count)
        print(f"Measured value: {measured} (over Q = {Q})")
        
        # Use continued fraction expansion to approximate the phase and recover the order r.
        _, r = continued_fraction_expansion(measured, Q)
        print(f"Estimated order r = {r}")
        
        # If r is odd, try another base.
        if r % 2 != 0:
            print("r is odd; retrying...\n")
            continue
        
        # Compute a^(r/2) mod N.
        exponent = r // 2
        factor_candidate = modexp(a, exponent, N)
        
        # Check that factor_candidate is non-trivial.
        if factor_candidate == N - 1 or factor_candidate == 1:
            print("Trivial factor encountered; retrying...\n")
            continue
        
        # Use gcd to attempt to extract the factors.
        factor1 = gcd(factor_candidate + 1, N)
        factor2 = gcd(factor_candidate - 1, N)
        
        if factor1 not in [1, N]:
            return factor1, N // factor1
        if factor2 not in [1, N]:
            return factor2, N // factor2
        
        print("Failed to obtain non-trivial factors; retrying...\n")
    
    return None

# ----------------- Main Execution: Factor the List of Semiprimes -----------------

# Dictionary mapping bit-size to semiprime number.
import semiprimes

results = {}

print("Starting Shor's algorithm on the list of semiprimes...\n")
for bit_size, N in semiprimes.items():
    print(f"---\nFactoring {N} (bit-size {bit_size})")
    # Adjust n_count based on the bit-size; here we use roughly half the bit-size plus one.
    factors = shor_factor(N, max_attempts=10, n_count=bit_size // 2 + 1)
    if factors:
        p, q = factors
        print(f"Success: {N} = {p} * {q}\n")
        results[N] = (p, q)
    else:
        print(f"Failed to factor {N} after several attempts.\n")
        
print("Final results:")
for N, (p, q) in results.items():
    print(f"{N} = {p} * {q}")
